In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import tensorflow as tf
import cv2

### MAPPING the Train and Test Folders

In [ ]:
trainData = '/kaggle/input/manualdataceleb/train'
testData = '/kaggle/input/manualdataceleb/test'# paths for traning and testing dataset within Kaggle working directory
dnnModel = 'hogModel.h5'# deep learning classifier target model name

In [ ]:
import os
lab_list = os.listdir(trainData)
print(lab_list) 
# for getting the list labelling of the dataset; a verification step
# this module can be run when inside the Kaggle working directory

In [ ]:
# variable declaration
imgW = 512
imgH = 512

# image  pixel depth. Important for input Neuron definitions. 
# Change the image definitions for processed image to say 128 X 128

trainSamples = 200
testSamples = 50

'''
train vs test. This has to be changed according to the image set and 
training versus testing data bifurcation

'''
epochs = 10
batchSize = 10
# signifies the training epoch and batch size. Batch size is increased according to the data size
# total size/batch size = runs during model generation 

Further there will be THREE models - ONLY one is used

###  Keras DNN

In [ ]:
'''
FOR DNN model

'''

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


def createModel(input_shape):
    
    model = Sequential()
    
    model.add(Flatten(input_shape = input_shape))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5))
    model.add(Activation('sigmoid'))
    
    model.compile(optimizer = 'rmsprop', 
                  loss = 'CategoricalCrossentropy',
                  metrics = ['accuracy'])
    
    return model

def train_model():
        
    datagen = ImageDataGenerator(rescale=1./255)
    
    model = createModel((512,512,3))
    print(model)
    
    train_generator = datagen.flow_from_directory(
            trainData,
            target_size = (imgW, imgH),
            batch_size = batchSize,
            shuffle = False)
    
    test_generator = datagen.flow_from_directory(
            testData,
            target_size = (imgW, imgH),
            batch_size = batchSize,
            shuffle = False)
    
    model.fit_generator(
            train_generator,
            steps_per_epoch = trainSamples, 
            epochs = epochs,
            validation_data = test_generator,
            validation_steps = trainSamples
            )
    model.summary()
    model.save(dnnModel)
    model_json = model.to_json()
    with open("/kaggle/working/finalModel.json", "w") as json_file:
        json_file.write(model_json)
train_model()

###  Keras CNN

In [ ]:
'''

KERAS CNN

'''

def createModel(input_shape):
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

def train_model():
        
    datagen = ImageDataGenerator(rescale=1./255)
    
    model = createModel((512,512,3))
    print(model)
    
    train_generator = datagen.flow_from_directory(
            trainData,
            target_size = (imgW, imgH),
            batch_size = batchSize,
            shuffle = False)
    
    test_generator = datagen.flow_from_directory(
            testData,
            target_size = (imgW, imgH),
            batch_size = batchSize,
            shuffle = False)
    
    model.fit_generator(
            train_generator,
            steps_per_epoch = trainSamples, 
            epochs = epochs,
            validation_data = test_generator,
            validation_steps = trainSamples
            )
    model.summary()
    model.save(dnnModel)
    model_json = model.to_json()
    with open("/kaggle/working/finalModel.json", "w") as json_file:
        json_file.write(model_json)
train_model()

### TF Keras CNN

In [ ]:
'''

TF Keras - CNN

'''

from tensorflow.keras.preprocessing.image import ImageDataGenerator

rescaled = ImageDataGenerator(rescale=1.0 / 255,
    rotation_range=5,
    zoom_range=(0.95, 0.95),
    horizontal_flip=False,
    vertical_flip=False,
    data_format="channels_first",
    validation_split=0.0,
    dtype=tf.float32)
train_fed = tf.keras.utils.image_dataset_from_directory('/kaggle/input/manualdataceleb/train', image_size = (512,512), batch_size=8, label_mode = 'categorical')
test_fed = tf.keras.utils.image_dataset_from_directory('/kaggle/input/manualdataceleb/test', image_size = (512,512), batch_size=8, label_mode = 'categorical')


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, (3,3), activation='relu', input_shape=(512,512,3)),
    tf.keras.layers.Conv2D(8, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(8, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),  #1st layer
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),  #2nd layer
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    #tf.keras.layers.MaxPool2D(2,2),  #3rd layer
    #tf.keras.layers.Dropout(0.2),
    #tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    #tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    #tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),  #4th layer
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5, activation='softmax')                                                       
])

def train_model():
        
    datagen = ImageDataGenerator(rescale=1./255)
    
    model = createModel((512,512,3))
    print(model)
    
    train_generator = datagen.flow_from_directory(
            trainData,
            target_size = (imgW, imgH),
            batch_size = batchSize,
            shuffle = False)
    
    test_generator = datagen.flow_from_directory(
            testData,
            target_size = (imgW, imgH),
            batch_size = batchSize,
            shuffle = False)
    
    model.fit_generator(
            train_generator,
            steps_per_epoch = trainSamples, 
            epochs = epochs,
            validation_data = test_generator,
            validation_steps = trainSamples
            )
    model.summary()
    model.save(dnnModel)
    model_json = model.to_json()
    with open("/kaggle/working/finalModel.json", "w") as json_file:
        json_file.write(model_json)
train_model()

In [ ]:
# compiler for TF Keras
from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Run TF Keras
history = model.fit(train_fed, shuffle=True, epochs=20, validation_data=test_fed, verbose=1)

## IMAGE PRE-PROCESSING

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time 
%matplotlib inline

In [ ]:
def convertToRGB(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#### HAAR Cascade and HOG filter

In [ ]:
haar_face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_alt.xml')
haar_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
haar_eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')


import os
from pathlib import Path
from PIL import Image
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
import matplotlib.pyplot as plt

basepath = '/kaggle/input/facesceleb'

pathArr = []
fpathArr = []

with os.scandir(basepath) as entries:
    for entry in entries:
        if entry.is_dir():
            #print(entry.name)
            folder = Path(entry.path)
            files_in_entry = folder.iterdir()
            for item in files_in_entry:
                if item.is_file():
                    #print("\t"+item.name)
                    #print(os.getcwd())
                    pathArr.append(entry.path)
                    fpathArr.append(entry.path+"/"+item.name)

#print(pathArr)

for i in range(1,len(fpathArr)):
    print(fpathArr[i]+"...")
    fPath = fpathArr[i]
    face_img = cv2.imread(fPath)
    
    

    gray_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)

    #plt.imshow(gray_img, cmap='gray')
    
    #plt.imshow(gray_img, cmap='gray')
    
    #plt.show()

    faces = haar_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=1)
    
    cropped_image = face_img
    
    
    
    
    if(len(faces) == 1):
        print('Face found')
        
        for (x, y, w, h) in faces:
            cv2.rectangle(face_img, (x, y), (x+w, y+h), (0, 255, 0), 2)

        #plt.imshow(convertToRGB(test1))

        #plt.show()

        cropped_image = face_img[y:y+h, x:x+w]

        #plt.imshow(convertToRGB(cropped_image))

        #plt.show()
    
        #plt.axis("off")
        #plt.imshow(img)
        #print(img.shape)

        resized_img = resize(cropped_image, (128*4, 64*4))
        plt.axis("off")
        #plt.imshow(resized_img)
        #plt.show()
        #print(resized_img.shape)

        #creating hog features
        fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                	cells_per_block=(2, 2), visualize=True, multichannel=True)
        #print(fd.shape)
        #print(hog_image.shape)
        plt.axis("off")
        #plt.imshow(hog_image, cmap="gray")
        #plt.show()
        
        pathArr[i] = pathArr[i].replace("input/facesceleb","working/facesceleb")
    
        if not os.path.exists(pathArr[i]):
            os.makedirs(pathArr[i])
    
        
        fPath = fPath.replace("input/facesceleb","working/facesceleb")
        #plt.imsave("resized_img.jpg", resized_img)
        plt.imsave(fPath, hog_image, cmap="gray")
    else:
        #fPath = fPath.replace("input/facesceleb","working/facesceleb")
        pathArr[i] = pathArr[i].replace("input/facesceleb","working/facenotfound")
        
        if not os.path.exists(pathArr[i]):
            os.makedirs(pathArr[i])
        
        fPath = fPath.replace("input/facesceleb","working/facenotfound")
        
        print('check')
        
        print(fPath)
        
        #cv2.imwrite(fPath, face_img)
        
        #face_img.save(fPath)
        
        plt.imsave(fPath, face_img)
    

In [ ]:
'''
downloading from Kaggle ZIP file 
Kaggle will not allow folder download 

'''
import shutil
shutil.make_archive("facenotfound", 'zip', "/kaggle/working/facenotfound")

After Train Test split Upload the files as Zip file into new folder and follow to the
#### MAPPING the Train and Test Folders section 

and then run the same deep learning model as attempted in the previous step, say example

### TF Keras CNN

### Code segment For Train Test Split

In [ ]:
import os
from pathlib import Path

basepath = '/kaggle/input/facedata/train'
with os.scandir(basepath) as entries:
    for entry in entries:
        if entry.is_dir():
            print(entry.name)
            entry = Path(entry.path)
            files_in_entry = entry.iterdir()
            for item in files_in_entry:
                if item.is_file():
                    print("\t"+item.name)

## Generating Sample Images

In [ ]:
face_img = cv2.imread('/kaggle/input/facesceleb/FacesAlexDaddario/A24.jpg')

print("Original Image")

plt.imshow(convertToRGB(face_img))

plt.show()

print("GrayScale Image")

gray_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)

plt.imshow(gray_img, cmap='gray')

plt.show()

#plt.imshow(gray_img, cmap='gray')


In [ ]:
#let's detect multiscale images
faces = haar_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=1)

#print the number of faces found
if(len(faces)>0):
    print('Faces found=',len(faces))

In [ ]:
for (x, y, w, h) in faces:
    cv2.rectangle(face_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
from PIL import Image

print("Image with Face Rectangle")

plt.imshow(convertToRGB(face_img))

plt.show()

print("Cropped Face Image")
cropped_image = face_img[y:y+h, x:x+w]

plt.imshow(convertToRGB(cropped_image))

plt.show()

In [ ]:
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
%matplotlib inline


#reading the image
img = convertToRGB(cropped_image)
imshow(img)
print(img.shape)

In [ ]:
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
import matplotlib.pyplot as plt

#reading the image
#img = imread('/kaggle/input/facedata/test/1026/13432.jpg')
plt.axis("off")
plt.imshow(img)
print(img.shape)

#resizing image
print("Resized Image")
resized_img = resize(img, (128*4, 64*4))
plt.axis("off")
plt.imshow(resized_img)
plt.show()
print(resized_img.shape)

#creating hog features
fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                	cells_per_block=(2, 2), visualize=True, multichannel=True)
print(fd.shape)
print(hog_image.shape)

print("HOG Image")
plt.axis("off")
plt.imshow(hog_image, cmap="gray")
plt.show()

# save the images
plt.imsave("resized_img.jpg", resized_img)
plt.imsave("hog_image.jpg", hog_image, cmap="gray")